In [ ]:
## Standard imports + some extra imports for 3d plots
from matplotlib import pyplot
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

## Defining All functions here

# Tolerance Calculation
def L1norm(new, old):
    '''
    Calculates L1 Norm
    ----------
    Parameters:
    ----------
    new: 2D array of floats
        Current values to calculate L1 Norm
    old: 2D array of floats
        Original values to calculate L1 Norm 
    -------
    Returns:
    -------
    norm: float
        Calculated L1 Norm
    '''
    norm = numpy.sum(numpy.abs(new-old))
    return norm

# Poisson Relaxation
def poisson_2d(p, b, dx, dy):
    '''
    Performs  Poisson Jacobi relaxation for a single iteration
    ----------
    Parameters:
    ----------
    p: 2D array of floats
        Initial guess
    b: 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction    
    -------
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    norm: float
        L1norm after one iteration of relaxation
    '''
    pd = p.copy() # Dummy Variable used in calculating the new p and the norm, keeps original

    p[1:-1,1:-1] = 1/(2*(dx**2 + dy**2)) * \
        ((pd[1:-1,2:]+pd[1:-1,:-2])*dy**2 + \
        (pd[2:,1:-1] + pd[:-2,1:-1])*dx**2 - \
        b[1:-1,1:-1]*dx**2*dy**2)
    
    # BCs are automatically enforced - all zero dirichlet
    norm = L1norm(p,pd)
    return p, norm 

# Laplace Relaxation
def laplace_2d(p, p0, p1, dx, dy):
    '''
    Performs  Laplace Jacobi relaxation for a single iteration
    ----------
    Parameters:
    ----------
    p: 2D array of floats
        Initial guess
    p0: 2D vector of floats
          First row to create second order boundary
    p1: 2D vector of floats
        Second row to create second order boundary
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction    
    -------
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    norm: float
        L1norm after one iteration of relaxation
    '''    
    pn = p.copy()
    
    # Apply boundary conditions
    p[0,:] = p0
    p[1,:] = p1
    p[2.:] = func(p1.p0)
    
    # Relax p
    p[3:-1,1:-1] = .25 * (pn[3:-1,2:-1]+pn[3:-1,0:-2]+pn[4:-1.1:-1]+pn[2:-2,1:-1])


    norm = L1norm(p, pn)
    return p, norm

# Creating the plot3d method (I'd rather see the 3d then attempt a contour-can change viewing angle as needed)
def plot3d(x, y, p):
    '''
    Plots a 3D graph - Figure Appears with appropriate limits and viewing angle, No Return Values
    ----------
    Parameters:
    ----------
    x: array of float
        nodal coordinates in x
    y: array of float
        nodal coordinates in y
    p: 2D array of float
        calculated potential field
    '''
    fig = pyplot.figure(figsize=(10,8), dpi=1000)
    ax = fig.gca(projection='3d')
    X,Y = numpy.meshgrid(x,y)
    ax.plot_surface(X,Y,p[:], rstride=1, cstride=1, cmap=cm.viridis,
            linewidth=0, antialiased=False)
    
    # Cosmetics
    ax.set_xlim(0,1) # Domain given in problem
    ax.set_ylim(0,1) # Range given in problem
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.view_init(30,45)

# Creating funciton to get maximum value of an array
def getmax(val):
    '''
    Gets maximum of absolute values of all elements in an array
    ----------
    Parameters:
    ----------
    val:2D array of floats
        array to be maximized   
    -------
    Returns:
    -------
    max: float
        value of maximum absolute value 
    '''
    max = numpy.amax(numpy.absolute(val))
    return max

## Setting Up initial conditions
nx, ny = 41, 41 # Grid Size
l, h  = 1., 1. # Domain size
dx, dy = l/(nx-1), h/(ny-1) # Mesh Element size
l1_target = 1e-6 # Tolerance for solution

# Initializing solution arrays
psi = numpy.zeros((nx,ny))
w = numpy.zeros((nx,ny))

## Create/Simulate a Do While Loop
#  We want the solution to relax at least in one iteration so that we can caluculate the L1norm accurately.
#  This means we must have the loop run at least once. One way to do that is set an arbitrary conditional
#  boolean that we can then update based on our L1norm calculations after each iteration!
## <End Comment Block>

d_bool = True
while d_bool:
    